In [11]:
from retraction_analysis.utils import get_retractions

r = get_retractions()

In [44]:
import plotly.graph_objects as go
import pandas as pd
import psycopg

# CIViC

In [2]:
from civicpy import civic

In [3]:
evidence = civic.get_all_evidence()

In [37]:
civic_pmids = {i.source.citation_id for i in evidence}

In [38]:
retracted_pmids = {str(int(pmid)) for pmid in r["RetractionPubMedID"].dropna()} - {0}

In [39]:
retracted_pmids.intersection(civic_pmids)

set()

✅

# DGIdb

Focusing on v5 claims initially

In [45]:
with psycopg.connect("dbname=dgidb user=postgres") as conn:
    with conn.cursor() as cur:
        result = cur.execute("SELECT * FROM publications")
        pubs = result.fetchall()

In [53]:
dgidb_pmids = {str(i[1]) for i in pubs}
len(dgidb_pmids)

12239

In [52]:
retracted_pmids.intersection(dgidb_pmids)

{'15899874'}

❌

In [59]:
with psycopg.connect("dbname=dgidb user=postgres") as conn:
    with conn.cursor() as cur:
        result = cur.execute("""
        SELECT s.source_db_name, 
        dc.name,
        gc.name,
        p.citation

        FROM interaction_claims ic
        LEFT JOIN sources s on ic.source_id = s.id
        JOIN interaction_claims_publications icp ON icp.interaction_claim_id = ic.id
        JOIN publications p ON icp.publication_id = p.id
        LEFT JOIN drug_claims dc ON ic.drug_claim_id = dc.id
        LEFT JOIN gene_claims gc ON ic.gene_claim_id = gc.id 
        WHERE p.pmid = 15899874;
        """)
        print(result.fetchall())

[('NCI', '5-FLUORODEOXYURIDINE', 'DNMT1', 'Ghoshal K et al., 2005, 5-Aza-deoxycytidine induces selective degradation of DNA methyltransferase 1 by a proteasomal pathway that requires the KEN box, bromo-adjacent homology domain, and nuclear localization signal., Mol Cell Biol')]
